### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [ ]:
# poetry add python-dotenv langchain langchain-openai

In [4]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    # model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
 model="openai/gpt-oss-120b",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
print(response)

('ChatGPT는 대량의 텍스트 데이터를 사용해 **자기지도학습**(self‑supervised learning) 방식으로 '
 '사전학습(pre‑training)을 수행합니다. 이 과정에서 모델은 다음에 올 단어를 예측하도록 학습하면서 언어의 통계적 패턴과 의미 '
 '관계를 내부 파라미터에 압축합니다. 이후에는 인간이 만든 질문‑답변 쌍 등으로 **지도학습**(fine‑tuning)과 '
 '**강화학습**(RLHF)을 적용해, 보다 정확하고 안전한 대화 능력을 갖추게 됩니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [24]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**ChatGPT 모델의 학습 원리 (3문장)**  \n'
 '1. 대규모 텍스트 데이터를 사용해 토큰 간의 확률적 관계를 학습하고, 다음에 올 단어를 예측하도록 사전 학습(pre‑training)을 '
 '진행합니다.  \n'
 '2. 사전 학습된 가중치를 기반으로, 인간이 만든 질문‑답변 쌍이나 대화 로그 등으로 모델의 응답 품질을 높이는 지도 '
 '학습(supervised fine‑tuning)과, 모델 자체가 생성한 답변을 평가해 보상 신호를 주는 강화 학습(RLHF)을 '
 '수행합니다.  \n'
 '3. 최종적으로 여러 단계의 미세 조정을 거쳐, 다양한 주제와 문맥에 맞는 자연스러운 텍스트를 생성하도록 최적화됩니다.  \n'
 '\n'
 '**ChatGPT 모델의 장점 요약**  \n'
 '- **범용성**: 광범위한 분야의 지식과 언어 패턴을 학습해 다양한 질문에 유연하게 대응한다.  \n'
 '- **대화 흐름 유지**: 문맥을 기억하고 이전 발언을 고려해 일관된 대화를 이어갈 수 있다.  \n'
 '- **빠른 응답**: 대규모 병렬 연산과 최적화된 추론 구조 덕분에 실시간에 가까운 속도로 텍스트를 생성한다.  \n'
 '\n'
 '**ChatGPT와 비슷한 AI 모델 (영어 모델명)**  \n'
 '- GPT‑4 (OpenAI)  \n'
 '- LLaMA 2 (Meta)  \n'
 '- Claude (Anthropic)  \n'
 '- Gemini (Google DeepMind)  

#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [25]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemini", "count": 3},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.
('GPT-4는 대규모 텍스트 데이터를 이용해 다음에 올 단어를 예측하도록 하는 **자기지도학습**(self‑supervised '
 'learning) 방식으로 사전 학습을 진행합니다. 이후에는 인간이 만든 질문‑답변 쌍이나 피드백을 활용해 **강화학습**(RLHF)으로 '
 '모델의 출력 품질과 안전성을 조정합니다.')
<class 'str'> Gemini 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('Gemini 모델은 대규모 텍스트와 멀티모달 데이터(이미지, 오디오 등)를 사용해 사전 학습(pre‑training)된 거대 언어·비전 '
 '모델입니다. 사전 학습 단계에서는 자기지도 학습(self‑supervised learning)과 대규모 샘플링을 통해 입력과 출력 사이의 '
 '확률적 관계를 학습하고, 이후 특정 작업에 맞춰 소량의 라벨 데이터로 미세 조정(fine‑tuning)합니다. 최종적으로는 인간 피드백을 '
 '활용한 강화학습(RLHF) 등을 적용해 응답 품질과 안전성을 높입니다.')


##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [10]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 오픈AI사가 개발한 인공지능 챗봇입니다. ChatGPT의 학습 원리를 설명해 드리겠습니다.

ChatGPT는 대규모 언어 모델(LLM, Large Language Model)로서, 자연어 처리(NLP, Natural Language Processing) 분야에서 널리 사용되는 기술입니다. ChatGPT는 다음과 같은 원리로 학습합니다.

1. **데이터 수집**: ChatGPT는 인터넷에서 수집된 대규모 텍스트 데이터를 학습합니다. 이 데이터는 책, 기사, 블로그 포스트, 소셜 미디어 등 다양한 소스에서 수집됩니다.

2. **토큰화**: 수집된 텍스트 데이터는 토큰이라는 단위로 분할됩니다. 토큰은 단어, 구절, 또는 기호일 수 있습니다.

3. **임베딩**: 각 토큰은 벡터 공간에 임베딩됩니다. 임베딩은 토큰의 의미를 벡터로 표현하는 과정입니다. 이 벡터는 토큰의 의미를 나타내는 숫자 값으로 구성됩니다.

4. **트랜스포머**: ChatGPT는 트랜스포머라는 신경망 구조를 사용합니다. 트랜스포머는 입력 토큰의 임베딩을 처리하여 출력 토큰의 임베딩을 생성합니다.

5. **마스킹된 언어 모델링**: ChatGPT는 마스킹된 언어 모델링이라는 학습 목표를 사용합니다. 이 목표는 입력 토큰의 일부를 무작위로 가리고, 가려진 토큰을 예측하는 것입니다.

6. **다음 토큰 예측**:

In [ ]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [11]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

 I'm happy to provide more information on the topic.

**Deep Learning** is a subset of **Machine Learning** (ML) that involves the use of **Artificial Neural Networks** (ANNs) to analyze data. Inspired by the structure and function of the human brain, ANNs are composed of layers of interconnected nodes or **neurons** that process and transform inputs into meaningful representations.

**Key Characteristics:**

1. **Multiple Layers**: Deep learning models have multiple layers of neurons, which allow them to learn complex patterns and relationships in data. These layers are typically organized in a hierarchical manner, with early layers learning low-level features and later layers learning high-level features.
2. **Hierarchical Representations**: Each layer in a deep learning model learns to represent the input data in a more abstract and meaningful way. This hierarchical representation enables the model to capture complex patterns and relationships in the data.
3. **Distributed Represent

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [12]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 좋아해요.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [26]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
## 태양계 행성 간략 정리

| 순서 | 행성 | 특징·주요 정보 |
|------|------|----------------|
| 1 | **수성 (Mercury)** | - 태양에 가장 가까운 행성<br>- 평균 직경 ≈ 4,880 km (지구의 38%)<br>- 대기가 거의 없으며, 낮과 밤 온도 차가 극심 (−173 °C ~ +427 °C) |
| 2 | **금성 (Venus)** | - “지구의 쌍둥이”라 불리지만, 두꺼운 이산화탄소 대기와 황산 구름으로 온실효과가 강함<br>- 표면 온도 ≈ 467 °C (지구보다 훨씬 뜨거움)<br>- 자전 방향이 역행(시계 반대) |
| 3 | **지구 (Earth)** | - 물이 액체 상태로 존재하는 유일한 알려진 행성<br>- 평균 직경 ≈ 12,742 km<br>- 대기 구성: N₂ ≈ 78%, O₂ ≈ 21% |
| 4 | **화성 (Mars)** | - “붉은 행성”이라 불리는 이유는 철산화물(녹) 때문에<br>- 평균 직경 ≈ 6,779 km (지구의 절반)<br>- 얇은 CO₂ 대기, 극지에 물 얼음 존재 |
| 5 | **목성 (Jupiter)** | - 태양계에서 가장 큰 행성 (직경 ≈ 139,820 km)<br>- 주성분은 수소·헬륨, 강력한 자기장 보유<br>- 79개의 위성(2025년 현재) 중 갈릴레오 위성(이오, 유로파, 가니메데, 칼리스토) 유명 |
| 6 | **토성 (Saturn)** | - 아름다운 고리 시스템으로 유명 (주로 얼음·암석 파편)<br>- 직경 ≈ 116,460 km, 밀도가 물보다 낮아 물에 뜰 수 있음<br>- 위성 80여 개, 타이탄은 두꺼운 대기와 메탄 호수를 가짐 |
| 7 | **천왕성 (Uranus)** | - 청록색을 띤 메탄 가스 풍부<br>- 자전축이 약 98°로 거의 옆으로 누워 있어 계절 변화가 극단적<br>- 직경 ≈ 50,724 km, 위성 27개(티탄i

##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [21]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 유치원생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 커뮤니케이터입니다. 아이에게 말하는 듯한 부드러운 말투를 가지죠"),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성은 뭐가 있어?"})
print(result.content)

print(llm)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 유치원생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 커뮤니케이터입니다. 아이에게 말하는 듯한 부드러운 말투를 가지죠'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=Prom

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [27]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 나타나는 대표적인 지구과학 현상은 **태풍**도 맞지만, 가을은 대기·해양·지표면이 서로 크게 변동하는 시기이기 때문에 여러 가지 다른 현상도 동시에 활발히 일어납니다. 아래에서는 **가을에 주로 발생하는 지구과학 현상 3가지**를 간략히 정리하고, 각각이 왜 가을에 두드러지는지 설명드릴게요.

---

## 1️⃣ 태풍 (열대 저기압) – “가을형 태풍”
| 특징 | 가을에 두드러지는 이유 |
|------|----------------------|
| **발달 조건**: 따뜻한 해수면(≥ 26 °C), 높은 수분 함량, 저기압 중심의 발달 | 가을이 되면 **동아시아·서태평양**의 해수면 온도가 여름 최고치를 유지하면서도, **동쪽으로 이동하는 제트기류**가 강해져서 태풍이 북쪽(한반도·일본)으로 빠르게 이동·전개됩니다. |
| **경로**: 서쪽으로 이동 → 북동쪽으로 회전 | 계절풍이 남쪽에서 북쪽으로 바뀌는 **동풍대(동아시아·북태평양 제트기류)**가 형성돼, 태풍이 남쪽에서 북쪽으로 ‘전환’하면서 한반도·동북아 지역에 접근합니다. |
| **피크 시기**: 9월 ~ 10월 초 | 이 시기에 해수면 온도가 아직 높고, 대기 불안정성도 강해 태풍 발생 빈도가 최고조에 달합니다. |

> **핵심 정리**: 가을은 “태풍이 가장 강하고, 북쪽으로 이동하기 쉬운 시기”이므로, 가을형 태풍이 대표적인 가을 현상이라 할 수 있습니다.

---

## 2️⃣ 중위도 저기압·전선성 폭풍 (시베리아 고기압·동아시아 저기압 전이)
| 특징 | 가을에 두드러지는 이유 |
|------|----------------------|
| **전형적인 형태**: 차가운 시베리아 고기압이 남하하면서 **동아시아·북태평양 저기압**(중위도 저기압)과 만나 전선이 형성됨 | 가을에는 **시베리아 고기압이 점차 강해져 남하**하고, 남쪽

In [30]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season = get_current_season("north")  # 계절 값 얻기
print(f"현재 계절: {season}")

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
# llm = ChatOpenAI(
#     #api_key=OPENAI_API_KEY,
#     base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
#     model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
#     temperature=0.0
# )

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season}에 발생하는 자연 현상:\n{response}")

현재 계절: 가을

 가을에 발생하는 자연 현상:
가을에 발생하는 대표적인 지구과학 현상 3가지를 알려드리겠습니다.

1.  **월식**: 지구가 태양과 달 사이를 지나면서 달에 그림자를 드리우는 현상입니다. 태양, 지구, 달이 일직선상에 위치할 때 발생하며, 태양의 빛이 달에 닿지 못하게 되어 달에 그림자가 발생합니다. 월식은 지구과학적으로 중력의 영향을 받으며, 달의 궤도가 지구의 궤도와 일치하지 않아서 발생합니다.
2.  **성운**: 별에서 뿜어져 나오는 가스나 먼지가 모여 형성된 구름 모양의 천체를 말합니다. 성운은 별의 생성과 소멸 과정에서 발생하며, 별의 생명을 연장하는 역할을 합니다. 성운은 주로 가을에 발생하며, 성운의 종류로는 성간 물질, 별의 잔해, 가스 구름 등이 있습니다.
3.  **태양의 황혼**: 태양의 천정에 비가 내리면서 태양이 지는 하늘에 발생하는 붉은색의 구름 현상입니다. 태양의 황혼은 대기 중의 물방울이나 얼음 결정에 의해 발생하며, 태양의 빛이 산란되어 붉은색으로 나타납니다. 태양의 황혼은 가을에 발생하는 현상으로, 대기 중의 수분 함량이 높을 때 발생합니다.

이러한 현상들은 지구과학적으로 중요한 의미를 가지며, 자연의 아름다움을 경험할 수 있는 기회입니다.


#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [ ]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)